In [1]:
import numpy as np
from pathlib import Path
import os, cv2
from keras.models import Sequential, Model
from keras.layers import concatenate, Dense, Conv2D, Flatten, Input
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

Using TensorFlow backend.


In [2]:
path = Path(os.path.abspath(''))

In [3]:
path

WindowsPath('C:/Users/patry/OneDrive/Pulpit/Sieci neuronowe/Music-instrument-recognition/Networks')

SPECTOGRAM PART

In [13]:
train_x = np.load(str(path) + '\\train_x.npy')
train_y = np.load(str(path) + '\\train_y.npy')

In [17]:
train_x = train_x.reshape(6705,128,130,1)

In [18]:
trainInput = Input(shape=(128,130,1))

x = Conv2D(128, kernel_size=3, activation='relu')(trainInput)
x = Conv2D(64, kernel_size=3, activation='relu')(x)
x = Conv2D(32, kernel_size=3, activation='relu')(x)
x = Flatten()(x)
x = Dense(11, activation='sigmoid')(x)

In [19]:
xModel = Model(inputs=trainInput, outputs=x)

PARAMETERS PART

In [20]:
train_x_params = np.load(str(path) + '\\train_x_params.npy')
train_y_params = np.load(str(path) + '\\train_y_params.npy')

In [36]:
# check if result arrays are the same

comparison = train_y_params == train_y
equal_arrays = comparison.all()
print(equal_arrays)

True


In [22]:
trainInput2 = Input(shape=(25,))

y = Dense(128, activation='relu')(trainInput2)
y = Dense(64, activation='relu')(y)
y = Dense(32, activation='relu')(y)
y = Dense(11, activation='sigmoid')(y)

In [27]:
yModel = Model(inputs=trainInput2, outputs=y)

In [29]:
combined = concatenate([xModel.output, yModel.output])
z = Dense(4, activation="relu")(combined)
z = Dense(1, activation="linear")(z)

In [30]:
model = Model(inputs=[xModel.input, yModel.input], outputs=x)
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [38]:
fit = model.fit([train_x, train_x_params], train_y, epochs=3)

Epoch 1/3
6705/6705 [==============================] - 723s 108ms/step - loss: 0.3582 - accuracy: 0.8887
Epoch 2/3
6705/6705 [==============================] - 999s 149ms/step - loss: 0.2502 - accuracy: 0.9083
Epoch 3/3
6705/6705 [==============================] - 1086s 162ms/step - loss: 0.1421 - accuracy: 0.9471


In [39]:
model.reset_metrics()

In [48]:
model.save('Models/model_both.h5')